In [37]:
import os
import json

import numpy as np
import pandas as pd
from datasets import Dataset, DatasetDict, Features, Value, load_from_disk
from transformers import AutoTokenizer

from rank_bm25 import BM25Okapi
from retrieval import SparseRetrieval

from utils_taemin import run_sparse_retrieval

In [8]:
model_name = 'klue/roberta-large'
tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenize_fn = tokenizer.tokenize

dataset_path = "./data/"
context_path = "wikipedia_documents.json"
with open(os.path.join(dataset_path, context_path), "r", encoding="utf-8") as f:
    wiki = json.load(f)

contexts = list(
    dict.fromkeys([v["text"] for v in wiki.values()])
)

In [9]:
contexts

['이 문서는 나라 목록이며, 전 세계 206개 나라의 각 현황과 주권 승인 정보를 개요 형태로 나열하고 있다.\n\n이 목록은 명료화를 위해 두 부분으로 나뉘어 있다.\n\n# 첫 번째 부분은 바티칸 시국과 팔레스타인을 포함하여 유엔 등 국제 기구에 가입되어 국제적인 승인을 널리 받았다고 여기는 195개 나라를 나열하고 있다.\n# 두 번째 부분은 일부 지역의 주권을 사실상 (데 팍토) 행사하고 있지만, 아직 국제적인 승인을 널리 받지 않았다고 여기는 11개 나라를 나열하고 있다.\n\n두 목록은 모두 가나다 순이다.\n\n일부 국가의 경우 국가로서의 자격에 논쟁의 여부가 있으며, 이 때문에 이러한 목록을 엮는 것은 매우 어렵고 논란이 생길 수 있는 과정이다. 이 목록을 구성하고 있는 국가를 선정하는 기준에 대한 정보는 "포함 기준" 단락을 통해 설명하였다. 나라에 대한 일반적인 정보는 "국가" 문서에서 설명하고 있다.',
 '이 목록에 실린 국가 기준은 1933년 몬테비데오 협약 1장을 참고로 하였다. 협정에 따르면, 국가는 다음의 조건을 만족해야 한다.\n* (a) 영속적인 국민\n* (b) 일정한 영토\n* (c) 정부\n* (d) 타국과의 관계 참여 자격.\n특히, 마지막 조건은 국제 공동체의 참여 용인을 내포하고 있기 때문에, 다른 나라의 승인이 매우 중요한 역할을 할 수 있다.  이 목록에 포함된 모든 국가는 보통 이 기준을 만족하는 것으로 보이는 자주적이고 독립적인 국가이다. 하지만 몬테비데오 협약 기준을 만족하는지의 여부는 많은 국가가 논쟁이 되고 있는 실정이다. 또한, 몬테비데오 협약 기준만이 국가 지위의 충분한 자격이든 아니든, 국제법의 견해 차이는 존재할 수 있다. 이 물음에 대한 다른 이론에 대한 고리는 아래에서 볼 수 있다.\n\n위 기준에 논거하여 이 목록은 다음 206개 국가를 포함하고 있다.\n* 일반 국제 승인을 받은 195개 자주 국가.\n** 유엔 가입 국가 193개\n** 성좌의 명칭으로 유엔에서 국제 승인을 받은 국가: 

In [10]:
for i, sent in enumerate(contexts):
    if sent.startswith("베피콜롬보"):
        print(i)
        print(sent)
        break

test = contexts[i]

23174
베피콜롬보
베피콜롬보는 수성 탐사 계획 중 하나로 ESA와 JAXA가 공동으로 계획했다. 소형 탐사선 2기를 보유하고 있으며, 유럽(MPO)과 일본MMO)에서 각각 한 기씩 제공했으며, 또한 한 기는 사진을 찍고, 다른 한 기는 자기장을 연구하는 등 역할이 확실히 구별되어 있다. 

#태양 성운, 행성계에 있어서, 수성에 대해 연구해야 할 것은 무엇인가?
#왜 수성의 밀도는 다른 지구형 행성보다 높은가?
#수성의 핵은 액체인가? 고체인가?
#오늘날도 수성 구조는 활동적인가?
#금성과 화성, 달도 가지고 있지 못 한 작은 행성이 왜 자기장을 가지고 있는가?
#수성의 주 성분이 철임에도, 분광 관측으로는 발견되지 않았던 이유는 무엇인가?
#극점의 영구 동토에는 황 혹은 얼음이 존재하는가?
#외기권의 형성 원리는 무엇인가?
#이온층이 없는데도, 자기장과 태양풍이 어떻게 상호 작용을 하는가?
#수성의 자화(磁化)된 환경이 지구에서 관측되는 오로라, 밴 앨랜대, 자기 폭풍 등이 존재한다는 것을 암시하는가?
#공간의 왜곡으로 인한 수성의 근일점 변화가 일반상대성이론에 근거한 결과의 오차값을 더 줄일 수 있는가

매리너 10호나 메신저와 같이, 베피콜롬보는 금성과 지구에서 플라이바이를 사용할 예정이다. 특히, 태양 에너지 추진을 이용하여 달, 금성을 지나 수성에 느린 속도로 도달 할 전망이다. 이런 기술은 태양 중력의 영향을 최소화하여 수성에 접근하기 위해서는 필수적이다

베피콜롬보는 2018년 10월 경에 발사 되어, 2025년 12월 5일, 수성 궤도로 진입 할 예정이다. 그 후, 2년동안 수성에 대한 정보를 모으고 연구를 행할 것이다.


In [11]:
retriever = SparseRetrieval(tokenize_fn=tokenize_fn)

Lengths of unique contexts : 56737


In [12]:
retriever.get_sparse_embedding()

Token indices sequence length is longer than the specified maximum sequence length for this model (1133 > 512). Running this sequence through the model will result in indexing errors


Build passage embedding
(56737, 50000)
Embedding pickle saved.


In [13]:
datasets = pd.read_csv("./csv_data/test_data.csv")

In [19]:
queries = datasets["question"].tolist()

In [21]:
test = retriever.get_relevant_doc_bulk(queries, 40)

In [38]:
query = queries[0]
query_vec = retriever.tfidfv.transform([query])
result = query_vec * retriever.p_embedding.T
result = result.toarray()

sorted_result = np.argsort(result.squeeze())[::-1]
doc_score = result.squeeze()[sorted_result].tolist()[:40]
doc_indices = sorted_result.tolist()[:40]

In [42]:
query_vec

<1x50000 sparse matrix of type '<class 'numpy.float64'>'
	with 16 stored elements in Compressed Sparse Row format>

In [40]:
doc_indices

[23174,
 15321,
 39474,
 10327,
 55049,
 2084,
 39470,
 35325,
 35320,
 17232,
 22606,
 20220,
 11036,
 33816,
 17618,
 8915,
 32829,
 38446,
 38435,
 20215,
 13224,
 17160,
 44720,
 17155,
 15070,
 39469,
 17235,
 17233,
 17230,
 17910,
 39870,
 25612,
 39475,
 17229,
 35321,
 22205,
 12195,
 16606,
 17231,
 29069]

In [15]:
df = retriever.retrieve(datasets, topk=40)

[query exhaustive search] done in 4.354 s


Sparse retrieval: : 0it [00:00, ?it/s]

In [12]:
df

,question,id,context
0,유령'은 어느 행성에서 지구로 왔는가?,mrc-1-000653,베피콜롬보\n베피콜롬보는 수성 탐사 계획 중 하나로 ESA와 JAXA가 공동으로 계...
1,용병회사의 경기가 좋아진 것은 무엇이 끝난 이후부터인가?,mrc-1-001113,냉전 종식 이후 전 세계적으로 소규모의 끊임없는 국지 분쟁들이 생겨나고 강대국들의 ...
2,돌푸스에게 불특정 기간동안 하원이 잠시 쉬는 것을 건의 받았던 인물은?,mrc-0-002191,"1933년 3월, 투표 과정의 위법성에 대한 문제제기가 불거졌다. 당시 오스트리아 ..."
3,"마오리언어와 영어, 뉴질랜드 수화를 공식 언어로 사용하는 나라는?",mrc-0-003951,다언어화자\n다언어화자는 두 개 이상의 언어를 모국어처럼 사용 가능한 사람이다. 2...
4,디엔비엔푸 전투에서 보응우옌잡이 상대한 국가는?,mrc-1-001272,타이응우옌시(v=Thành phố Thái Nguyên|hn=城舗太原)는 베트남 북...
...,...,...,...
595,타입 2 가이아 메모리을 만든 집단은?,mrc-0-002989,"재단 X가 슈라우드(=소노자키 후미네)가 개발한 가이아 메모리의 테크놀로지를 분석,..."
596,장면이 정치보복에 반대하는 입장에서 처벌을 원치 않은 대상은?,mrc-0-001804,과거의 죄악에 대한 소급입법 적용을 반대하였다. 4.19 혁명 당시 사망한 사망자의...
597,"콜드게임 중 어떠한 계기로 인해 잠시 중단된 뒤, 익일에 게임이 진행되는 것은?",mrc-0-003411,"2005년(제56회 / 지바 롯데 마린스 대 한신 타이거스 1차전·10월 22일, ..."
598,제2캐나다기갑여단이 상륙한 곳은?,mrc-0-003436,주노 해변\n\n주노 해변은 쿠르쇨르메르의 양쪽으로 뻗은 5마일 정도의 해변이었다....


In [21]:
df.iloc[0].context

'베피콜롬보\n베피콜롬보는 수성 탐사 계획 중 하나로 ESA와 JAXA가 공동으로 계획했다. 소형 탐사선 2기를 보유하고 있으며, 유럽(MPO)과 일본MMO)에서 각각 한 기씩 제공했으며, 또한 한 기는 사진을 찍고, 다른 한 기는 자기장을 연구하는 등 역할이 확실히 구별되어 있다. \n\n#태양 성운, 행성계에 있어서, 수성에 대해 연구해야 할 것은 무엇인가?\n#왜 수성의 밀도는 다른 지구형 행성보다 높은가?\n#수성의 핵은 액체인가? 고체인가?\n#오늘날도 수성 구조는 활동적인가?\n#금성과 화성, 달도 가지고 있지 못 한 작은 행성이 왜 자기장을 가지고 있는가?\n#수성의 주 성분이 철임에도, 분광 관측으로는 발견되지 않았던 이유는 무엇인가?\n#극점의 영구 동토에는 황 혹은 얼음이 존재하는가?\n#외기권의 형성 원리는 무엇인가?\n#이온층이 없는데도, 자기장과 태양풍이 어떻게 상호 작용을 하는가?\n#수성의 자화(磁化)된 환경이 지구에서 관측되는 오로라, 밴 앨랜대, 자기 폭풍 등이 존재한다는 것을 암시하는가?\n#공간의 왜곡으로 인한 수성의 근일점 변화가 일반상대성이론에 근거한 결과의 오차값을 더 줄일 수 있는가\n\n매리너 10호나 메신저와 같이, 베피콜롬보는 금성과 지구에서 플라이바이를 사용할 예정이다. 특히, 태양 에너지 추진을 이용하여 달, 금성을 지나 수성에 느린 속도로 도달 할 전망이다. 이런 기술은 태양 중력의 영향을 최소화하여 수성에 접근하기 위해서는 필수적이다\n\n베피콜롬보는 2018년 10월 경에 발사 되어, 2025년 12월 5일, 수성 궤도로 진입 할 예정이다. 그 후, 2년동안 수성에 대한 정보를 모으고 연구를 행할 것이다. R. J. 셰퍼는 목격자의 증언을 검증하는 점검 목록을 제공한다. \n\n# 저술이 실제로 의미하는 것이 문자 그대로의 의미와 다른 것인가 ? 언어는 현재 사용되는 의미와 다른 것인가 ? 문장이 풍자적이지는 않은가 ? (즉, 말하는 것과 다른 의미를 갖고 있는 것은 아닌가 ?)\n# 저자는 보고하는 사항

In [22]:
k = 1
if k == 1:
    f = Features(
        {
            "context": Value(dtype="string", id=None),
            "id": Value(dtype="string", id=None),
            "question": Value(dtype="string", id=None),
        }
    )

datasets = DatasetDict({"validation": Dataset.from_pandas(df, features=f)})

In [6]:
retriever.tfidfv

TfidfVectorizer(max_features=50000, ngram_range=(1, 2),
                tokenizer=<bound method PreTrainedTokenizerFast.tokenize of PreTrainedTokenizerFast(name_or_path='klue/roberta-large', vocab_size=32000, model_max_len=512, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'bos_token': '[CLS]', 'eos_token': '[SEP]', 'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'})>)

In [7]:
tfidfv = retriever.tfidfv.fit_transform(contexts)

Token indices sequence length is longer than the specified maximum sequence length for this model (1133 > 512). Running this sequence through the model will result in indexing errors


In [8]:
tfidfv

<56737x50000 sparse matrix of type '<class 'numpy.float64'>'
	with 18951489 stored elements in Compressed Sparse Row format>

In [41]:
tokenized_contexts = [tokenize_fn(context) for context in contexts]
bm25 = BM25Okapi(tokenized_contexts)

In [78]:
query = queries[30]
query

'아델리 펭귄의 목숨에 지장을 주는 사람의 업종은 무엇인가요?'

In [79]:
tokenized_query = tokenize_fn(query)
tokenized_query

['아',
 '##델',
 '##리',
 '[UNK]',
 '목숨',
 '##에',
 '지장',
 '##을',
 '주',
 '##는',
 '사람',
 '##의',
 '업종',
 '##은',
 '무엇',
 '##인',
 '##가요',
 '?']

In [81]:
bm25.get_top_n(tokenized_query, contexts, n=40)

['호수에서 엄마 키사키 에리를 찾아낸 모리 란은 무심코 달려오지만, 엄마인 에리는 란에게 멈추라고 한다. 갑자기 총성이 들리고, 엄마인 에리는 땅바닥에 넘어진다. 그런 꿈을 꾼 란은 바로 엄마에게 안부 전화를 한다. 엄마 에리는 웃으면서 넘기지만, 란이 전화에서 "꿈으로 본 엄마는 지금보다 조금 젊었다"라고 말하니, 에리는 표정이 흐려진다. 코난은 소년 탐정단과 함께 항공 박물관으로 가서 아가사 히로시를 기다린다. 그 동안에 아유미가 코난과의 사랑을 점친다. 결과는 "A의 예감"이었다. 이 의미는 낡은 은어로 키스를 의미하지만 그 의미는 코난만 알고 있다. 다른 소년 탐정단 아이들은 다른 방향으로 지레 짐작을 한다. 이 와중에 박물관에서 비행기의 사진을 찍던 유명한 사진가를 만난다. 이 때, 모리 탐정은 잠시 탐정 사무소를 비우게 된다. 그 후, 형무소에서 가석방한 지 얼마 안 된 사람이 전화를 모리 탐정 사무소에 걸었지만 아무도 받지 않았다. 무엇인가 의미 깊은 분위기를 감돌고 있었다. 일주일 후에, 공원에서 조깅을 하던 메구레 경부가 누군가에게 저격당한다. 그 다음에는 키사키 에리가 독이 든 초콜릿을 남편 모리 탐정이 보낸 선물인 줄 알고 먹다가 쓰러진다. 다행히 병원에서 위세척을 받아 목숨을 건졌지만 분위기가 점점 심상치 않으며 그 다음으로는 아가사 히로시가 어떤 사람에게 석궁으로 저격당한다.  코난은 이 사건을 토대로 코고로 아저씨나 자신을 둘러싼 인물을 노린다고 생각한다. 그리고 범인은 가석방한 지 얼마 안 된 그 사람이라고 경찰은 밝혀낸다. 이 범행의 목적은 아마 자신을 체포한 모리 코고로에게 복수한다고 생각할 수 있지만, 정말로 그럴까? 과연 그 사람의 단독 범행인가? 시간이 지나자 마침내 사망자까지 발생하는데….',
 "브라질에서 돌아온 과학자 폴과 그 사촌인 아델은 결혼을 앞두고 있다. 아델은 감수성과 사랑이 넘치지만, 폴은 아델을 자신의 사회적 지위를 위해 소유해야 하는 물건, 또는 수단으로만 바라본다. 폴은 결혼을 위해 아델의 저택에 머무

In [174]:
import time 

from contextlib import contextmanager

@contextmanager
def timer(name):
    t0 = time.time()
    yield
    print(f"[{name}] done in {time.time() - t0:.3f} s")

class CustomBM25(BM25Okapi):
    def __init__(self, corpus, tokenizer):
        super().__init__(corpus, tokenizer)

    def get_relevant_doc(self, query, k):
        query_vec = self.tokenizer(query)
        result = self.get_scores(query_vec)
        sorted_result = np.argsort(result.squeeze())[::-1]
        doc_score = result.squeeze()[sorted_result].tolist()[:k]
        doc_indices = sorted_result.tolist()[:k]
        return doc_score, doc_indices
    
    def get_relevant_doc_bulk(self, queries, k):
        doc_scores = []
        doc_indices = []
        for query in queries:
            doc_score, doc_indice = self.get_relevant_doc(query, k)
            doc_scores.append(doc_score)
            doc_indices.append(doc_indice)
        return doc_scores, doc_indices

In [175]:
test = CustomBM25(contexts, tokenize_fn)

In [194]:
sample_test = test.get_relevant_doc(query, 10)
sample_test

([40.50870729544352,
  40.20372732907141,
  38.831149826010375,
  38.4966590130543,
  37.73520627495076,
  37.38009641787722,
  36.3959095842119,
  36.29477596079964,
  35.825877431924226,
  35.71832856367597],
 [15317, 51611, 21961, 23174, 33889, 42057, 33122, 16295, 615, 13357])

In [181]:
bulk_test = test.get_relevant_doc_bulk(queries, 10)

In [185]:
bulk_test[1][0]

[39471, 43271, 23174, 5978, 38434, 12176, 11036, 3660, 38446, 10327]

In [192]:
queries[5]

'단공류가 일반 포유류와 다르다는 것을 알 수 있는 신체 부위는?'

In [193]:
[contexts[i] for i in bulk_test[1][5]]

['단궁류\n\n단공류(單孔類, Monotremata)는 현존하는 포유류 가운데 원수아강에 속하는 유일한 목으로, 포유류에 속한 다른 동물들과 여러 특징들을 공유하고 있다. 이들은 몸에 털이 있고 새끼에게 수유하며 온혈동물이다. 단공류는 또한 포유류의 두 가지 주요 유형인 태반포유류 및 유대류와 확실히 구분되는 많은 특징들을 가지고 있다. 아마 가장 뚜렷한 차이라면, 태반 포유류는 살아 있는 새끼를 낳고 캥거루와 같은 유대류는 새끼가 태어나기 전에 짧은 기간 동안 어미의 몸에 달려 있는 주머니 속에서 새끼를 키우는데, 단공류는 알을 낳는다는 사실일 것이다. 또한 단공류는 다른 포유류와 종의 수, 체내 기간, 골격 형태의 면에서도 다르다. 어떤 면에서 그들은 포유류보다는 파충류에 가깝다.',
 "단공류는 포유류의 극히 적은 부분이다. 멸종된 단공류를 제외하면, 여기에 해당하는 종의 수는 많지 않다. 그 중 하나는 오리너구리로, 이 동물은 오리의 부리와 비버의 꼬리, 그리고 물갈퀴 달린 발을 지니고 있다. 또한 여러 종류의 바늘두더지도 있는데, 이 동물은 거의 개미핥기를 닮았지만 뾰족한 털로 덮여 있어 고슴도치처럼 보이기도 한다. 오리너구리와 마찬가지로 이 동물은 대부분 오스트레일리아에 서식하며, 그 외 지역에서는 파푸아뉴기니밖에 분포하지 않는다.\n\n단공류라는 이름은 이 동물들의 체내 기관이 특이하기 때문에 붙여진 것이다. 단공류의 영어 이름은 Monotreme인데 'mono'는 하나를 의미하고 'trema'는 구멍(孔)을 의미한다. 다른 포유류들은 배변, 배설, 생식에 필요한 구멍들을 각기 따로 가지고 있다. 그러나 단공류는 이런 기관들이 전부 '총배설강'이라는 하나의 구멍으로 연결되어 있다. 이런 구조 때문에 단공류는 실제로 다른 포유류보다는 파충류에 더 가까운 것처럼 보인다. 단공류의 걸음걸이 또한 파충류와 비슷하다. 다른 포유류와 달리, 단공류의 다리는 몸체 바로 밑이 아닌 측면에 붙어 있고 앞다리가 뒷다리보다 약간 짧다. 따라서 이들의 걸음걸이는 네 

In [131]:
retriever.tfidfv.transform([query])

<1x50000 sparse matrix of type '<class 'numpy.float64'>'
	with 23 stored elements in Compressed Sparse Row format>

In [168]:
tokenize_fn(queries)

TypeError: TextEncodeInput must be Union[TextInputSequence, Tuple[InputSequence, InputSequence]]

In [17]:
from __future__ import absolute_import, division, print_function, unicode_literals
import numpy as np
import scipy.sparse as sp
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.utils.validation import check_is_fitted
from sklearn.feature_extraction.text import _document_frequency

class BM25Transformer(BaseEstimator, TransformerMixin):
    """
    Parameters
    ----------
    use_idf : boolean, optional (default=True)
    k1 : float, optional (default=2.0)
    b : float, optional (default=0.75)

    References
    ----------
    Okapi BM25: a non-binary model - Introduction to Information Retrieval
    http://nlp.stanford.edu/IR-book/html/htmledition/okapi-bm25-a-non-binary-model-1.html
    """
    def __init__(self, use_idf=True, k1=2.0, b=0.75):
        self.use_idf = use_idf
        self.k1 = k1
        self.b = b

    def fit(self, X):
        """
        Parameters
        ----------
        X : sparse matrix, [n_samples, n_features]
            document-term matrix
        """
        if not sp.issparse(X):
            X = sp.csc_matrix(X)
        if self.use_idf:
            n_samples, n_features = X.shape
            df = _document_frequency(X)
            idf = np.log((n_samples - df + 0.5) / (df + 0.5))
            self._idf_diag = sp.spdiags(idf, diags=0, m=n_features, n=n_features)
        return self

    def transform(self, X, copy=True):
        """
        Parameters
        ----------
        X : sparse matrix, [n_samples, n_features]
            document-term matrix
        copy : boolean, optional (default=True)
        """
        if hasattr(X, 'dtype') and np.issubdtype(X.dtype, np.float):
            # preserve float family dtype
            X = sp.csr_matrix(X, copy=copy)
        else:
            # convert counts or binary occurrences to floats
            X = sp.csr_matrix(X, dtype=np.float64, copy=copy)

        n_samples, n_features = X.shape

        # Document length (number of terms) in each row
        # Shape is (n_samples, 1)
        dl = X.sum(axis=1)
        # Number of non-zero elements in each row
        # Shape is (n_samples, )
        sz = X.indptr[1:] - X.indptr[0:-1]
        # In each row, repeat `dl` for `sz` times
        # Shape is (sum(sz), )
        # Example
        # -------
        # dl = [4, 5, 6]
        # sz = [1, 2, 3]
        # rep = [4, 5, 5, 6, 6, 6]
        rep = np.repeat(np.asarray(dl), sz)
        # Average document length
        # Scalar value
        avgdl = np.average(dl)
        # Compute BM25 score only for non-zero elements
        data = X.data * (self.k1 + 1) / (X.data + self.k1 * (1 - self.b + self.b * rep / avgdl))
        X = sp.csr_matrix((data, X.indices, X.indptr), shape=X.shape)

        if self.use_idf:
            check_is_fitted(self, '_idf_diag', 'idf vector is not fitted')

            expected_n_features = self._idf_diag.shape[0]
            if n_features != expected_n_features:
                raise ValueError("Input has n_features=%d while the model"
                                 " has been trained with n_features=%d" % (
                                     n_features, expected_n_features))
            # *= doesn't work
            X = X * self._idf_diag

        return X

In [18]:
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from scipy import sparse


class BM25(object):
    def __init__(self, b=0.75, k1=1.6):
        self.vectorizer = TfidfVectorizer(norm=None, smooth_idf=False)
        self.b = b
        self.k1 = k1

    def fit(self, X):
        """ Fit IDF to documents X """
        self.vectorizer.fit(X)
        y = super(TfidfVectorizer, self.vectorizer).transform(X)
        self.avdl = y.sum(1).mean()

    def transform(self, q, X):
        """ Calculate BM25 between query q and documents X """
        b, k1, avdl = self.b, self.k1, self.avdl

        # apply CountVectorizer
        X = super(TfidfVectorizer, self.vectorizer).transform(X)
        len_X = X.sum(1).A1
        q, = super(TfidfVectorizer, self.vectorizer).transform([q])
        assert sparse.isspmatrix_csr(q)

        # convert to csc for better column slicing
        X = X.tocsc()[:, q.indices]
        denom = X + (k1 * (1 - b + b * len_X / avdl))[:, None]
        # idf(t) = log [ n / df(t) ] + 1 in sklearn, so it need to be coneverted
        # to idf(t) = log [ n / df(t) ] with minus 1
        idf = self.vectorizer._tfidf.idf_[None, q.indices] - 1.
        numer = X.multiply(np.broadcast_to(idf, X.shape)) * (k1 + 1)                                                          
        return (numer / denom).sum(1).A1
